In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# İmport Libraries

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df  = pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

# EDA

In [ ]:
del df['customerID']

In [ ]:
df.head()
df.dtypes

In [ ]:
df.TotalCharges.values

In [ ]:
df = df[df.TotalCharges!=' ']
df.shape

In [ ]:
df.TotalCharges = pd.to_numeric(df.TotalCharges)

In [ ]:
df[df.Churn=='No'].tenure


In [ ]:
plt.figure(figsize=(14,9))
sns.histplot(x='tenure',hue='Churn',data=df,multiple="dodge",bins=10,shrink=.7,palette='light:#5A9')

In [ ]:
plt.figure(figsize=(14,9))
sns.histplot(x='MonthlyCharges',hue='Churn',data=df,multiple="dodge",bins=10,shrink=.7,palette='light:#5A9');

# Data Preprocessing

In [ ]:
cat_col = []
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes=='object':
            cat_col.append(column)
            print(f'{column} : {df[column].unique()}')
print_unique_col_values(df)
print(cat_col)

In [ ]:
df.replace('No internet service','No',inplace=True)
df.replace('No phone service','No',inplace=True)

In [ ]:
print_unique_col_values(df)

In [ ]:
cat_col = ['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'PaperlessBilling',
 'Churn']
one_hot_col = ['InternetService','Contract','PaymentMethod']

In [ ]:
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder
lb = LabelEncoder()
for i in cat_col:
    df[i] = lb.fit_transform(df[i])



In [ ]:
df = pd.get_dummies(data=df,columns=one_hot_col)


In [ ]:
df.head(2)

In [ ]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
df.head(1)

# Model 

In [ ]:
X = df.drop('Churn',axis='columns')
y =df['Churn']

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=5)

In [ ]:
X_train.shape

In [ ]:
len(X_train.columns)

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(26,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
model.fit(X_train,y_train,epochs=100)

In [ ]:
model.evaluate(X_test,y_test)


In [ ]:
yp = model.predict(X_test)


In [ ]:
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')